1、	创建多线程，查看saySorry打印效果（和上课一致即可）

In [ ]:
#coding=utf-8
import threading
import time
def saySorry():
    print("亲爱的，我错了，我能吃饭了吗？")
    time.sleep(1)
    
if __name__ == "__main__":
    for i in range(5):
        t = threading.Thread(target=saySorry)
        t.start() #启动线程，即让线程开始执行

2、	创建两个多线程，主线程，两个子线程分别While True，查看CPU占有率

In [2]:
import threading
import time

def do_while():
    while True:
        print("我是子线程1，我在执行")
        time.sleep(1)

t1 = threading.Thread(target=do_while)
t1.start() #启动线程，即让线程开始执行
t2 = threading.Thread(target=do_while)
t2.start() #启动线程，即让线程开始执行
print("我是主线程，我在执行")


我是子线程，我在执行
我是主线程，我在执行


我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行
我是子线程，我在执行


3、	创建一个子线程work1，传递列表a，然后work1中给列表a添加元素，然后创建线程work2，查看列表是否发生改变

In [1]:
from threading import Thread
import time


def work1(nums):
    nums.append(44)
    print("----in work1---", nums)


def work2(nums):
    # 延时一会，保证t1 线程中的事情做完
    time.sleep(1)
    print("----in work2---", nums)


g_nums = [11, 22, 33]
t1 = Thread(target=work1, args=(g_nums,))
t1.start()
t2 = Thread(target=work2, args=(g_nums,))
t2.start()
# g_nums会被work1和work2共享，因为work1和work2都是对g_nums进行操作，所以会出现线程安全问题

----in work1--- [11, 22, 33, 44]


----in work2--- [11, 22, 33, 44]


4、	创建两个子线程，分别对同一个全局变量加1百万，查看最终结果是否为2百万，如果不为2百万，通过加解锁后，实现最终是2百万（电脑速度过快的同学，可以搞2千万）

In [3]:
import threading
import time
g_num = 0


def work1(num):
    global g_num
    for i in range(num):
        g_num += 1
    print("----in work1, g_num is %d---" % g_num)


def work2(num):
    global g_num
    for i in range(num):
        g_num += 1
    print("----in work2, g_num is %d---" % g_num)


print("---线程创建之前g_num is %d---" % g_num)
t1 = threading.Thread(target=work1, args=(int(1e7),))
t1.start()
t2 = threading.Thread(target=work2, args=(int(1e7),))
t2.start()
while len(threading.enumerate()) != 1:
    time.sleep(0.01)
print("2 个线程对同一个全局变量操作之后的最终结果是:%s" % g_num)


---线程创建之前g_num is 0---
----in work1, g_num is 10026759---
----in work2, g_num is 13413827---


KeyboardInterrupt: 

5、	制造死锁问题，并用ps查看线程状态

In [ ]:
#coding=utf-8
import threading
import time
class MyThread1(threading.Thread):
    def run(self):
        # 对mutexA 上锁
        mutexA.acquire()
        # mutexA 上锁后，延时1 秒，等待另外那个线程把mutexB 上锁
        print(self.name+'----do1---up----')
        time.sleep(1)
        # 此时会堵塞，因为这个mutexB 已经被另外的线程抢先上锁了
        mutexB.acquire()
        print(self.name+'----do1---down----')
        mutexB.release()
        # 对mutexA 解锁
        mutexA.release()

class MyThread2(threading.Thread):
    def run(self):
        # 对mutexB 上锁
        mutexB.acquire()
        # mutexB 上锁后，延时1 秒，等待另外那个线程把mutexA 上锁
        print(self.name+'----do2---up----')
        time.sleep(1)
        # 此时会堵塞，因为这个mutexA 已经被另外的线程抢先上锁了
        mutexA.acquire()
        print(self.name+'----do2---down----')
        mutexA.release()
        # 对mutexB 解锁
        mutexB.release()


mutexA = threading.Lock() 
mutexB = threading.Lock() 


if __name__ == '__main__':
    t1 = MyThread1()
    t2 = MyThread2()
    t1.start()
    t2.start()

6、	观察列表，元组，字典，集合是否是Iterator，Iterable

In [4]:
from collections.abc import Iterable

mylist = [1, 2, 3, 4, 5]
mytuple = (1, 2, 3, 4, 5)
mydict = {'name': 'zhangsan', 'age': 18}
isinstance(mylist, Iterable), isinstance(mytuple, Iterable), isinstance(mydict, Iterable)


(True, True, True)

7、	自制迭代器并使用next及for in的处理

In [ ]:
# coding=utf-8
from collections.abc import Iterator


class MyList(object):
    """自定义的一个可迭代对象"""

    def __init__(self):
        self.items = []

    def add(self, val):
        self.items.append(val)

    def __iter__(self):
        myiterator = MyIterator(self)
        return myiterator  # __iter__方法要返回一个迭代器


class MyIterator(object):
    """自定义的供上面可迭代对象使用的一个迭代器"""

    def __init__(self, mylist):
        self.mylist = mylist
        # current 用来记录当前访问到的位置
        self.current = 0

    def __next__(self):
        if self.current < len(self.mylist.items):
            item = self.mylist.items[self.current]
            self.current += 1
            return item
        else:
            raise StopIteration

    def __iter__(self):
        return self


if __name__ == '__main__':
    mylist = MyList()
    print(isinstance(mylist, Iterator))
    mylist.add(1)
    mylist.add(2)
    mylist.add(3)
    mylist.add(4)
    mylist.add(5)
    # print(next(mylist)) #为啥不行
    for num in mylist:
        print(num)
    myIterator = MyIterator(mylist)
    print(isinstance(myIterator, Iterator))
    print(next(myIterator))  # 使用next()函数的时候，调用的就是迭代器对象的__next__方法
